In [7]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

import joblib
import re

sns.set(style="whitegrid")


In [8]:
df = pd.read_csv("../02_data_cleaning/clean_data/cars_merged.csv")
print("Dataset shape:", df.shape)
df.head()

Dataset shape: (2496, 17)


,ref_no,title,make,model,year,mileage,engine_cc,transmission,fuel,seats,location,price_usd,total_price_usd,price_ugx,total_price_ugx,url,source
0,BY759022,2006 HONDA ACTY TRUCK,Honda,Acty,2006,111250.0,6500.0,MT,Petrol,2,Location,2370,397300.0,8532000,1.430280e+09,/honda/acty-truck/by759022/id/11910020/,BeForward
1,BY759024,2013 MITSUBISHI CANTER,Mitsubishi,Canter,2013,312401.0,29900.0,Semi AT,Diesel,3,Location,3920,774700.0,14112000,2.788920e+09,/mitsubishi/canter/by759024/id/11910065/,BeForward
2,CA462307,2014 TOYOTA HIACE VAN\n ...,Toyota,Hiace,2014,161352.0,29800.0,AT,Diesel,3,Location,5960,849200.0,21456000,3.057120e+09,/toyota/hiace-van/ca462307/id/12565207/,BeForward
3,CA740447,2011 TOYOTA WISH\n ...,Toyota,Wish,2011,154001.0,17900.0,AT,Petrol,7,Location,2430,454400.0,8748000,1.635840e+09,/toyota/wish/ca740447/id/12831915/,BeForward
4,CB026605,2017 HONDA FIT HYBRID\n ...,Honda,Fit,2017,111072.0,14900.0,AT,Hybrid(Petrol),5,Location,4790,663300.0,17244000,2.387880e+09,/honda/fit-hybrid/cb026605/id/13106276/,BeForward


In [9]:
# Keep relevant columns
df = df[[
    "make","model","year","mileage","engine_cc","seats",
    "transmission","fuel","source",
    "price_ugx"
]].copy()

# Remove missing prices
df = df[df["price_ugx"].notnull()]


In [10]:
def force_numeric(val):
    if pd.isna(val):
        return np.nan
    s = str(val).lower().strip()
    if s in ["ask", "-", "--", "", "nan", "unknown"]:
        return np.nan
    digits = re.sub(r"\D", "", s)
    return float(digits) if digits else np.nan

for col in ["mileage", "engine_cc", "year", "seats"]:
    df[col] = df[col].apply(force_numeric)

# Fill missing
df["mileage"].fillna(df["mileage"].median(), inplace=True)
df["engine_cc"].fillna(df["engine_cc"].median(), inplace=True)
df["year"].fillna(df["year"].median(), inplace=True)
df["seats"].fillna(4, inplace=True)


/var/folders/k7/ggnq5s_d0tg4z20frzwdzy_r0000gn/T/ipykernel_82542/4232595930.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["mileage"].fillna(df["mileage"].median(), inplace=True)
/var/folders/k7/ggnq5s_d0tg4z20frzwdzy_r0000gn/T/ipykernel_82542/4232595930.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting

In [11]:
# Compute median price per make+model+year
group_key = ["make","model","year"]
median_prices = df.groupby(group_key)["price_ugx"].median().reset_index()
median_prices.rename(columns={"price_ugx":"fair_price"}, inplace=True)

df = df.merge(median_prices, on=group_key, how="left")

# Define labels
def classify(row):
    if row["price_ugx"] > row["fair_price"] * 1.1:
        return "overpriced"
    elif row["price_ugx"] < row["fair_price"] * 0.9:
        return "underpriced"
    else:
        return "fair"

df["price_status"] = df.apply(classify, axis=1)

df["price_status"].value_counts()


price_status
fair           1670
overpriced      425
underpriced     401
Name: count, dtype: int64

In [12]:
features = [
    "make","model","year","mileage","engine_cc","seats",
    "transmission","fuel","source"
]

target = "price_status"

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

X_train.shape, X_test.shape


((1872, 9), (624, 9))

In [13]:
numeric = ["year","mileage","engine_cc","seats"]
categorical = ["make","model","transmission","fuel","source"]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical)
    ]
)

In [14]:
models = {
    "LogisticRegression": LogisticRegression(max_iter=500),
    "DecisionTree": DecisionTreeClassifier(),
    "RandomForest": RandomForestClassifier(n_estimators=200),
    "GradientBoosting": GradientBoostingClassifier()
}

results = {}

In [18]:
for name, model in models.items():
    print(f"\nTraining: {name}")

    clf = Pipeline(steps=[
        ("preprocess", preprocess),
        ("model", model)
    ])

    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)

    acc = accuracy_score(y_test, preds)
    print("Accuracy:", acc)
    print(classification_report(y_test, preds))

    results[name] = acc



Training: LogisticRegression
Accuracy: 0.655448717948718
              precision    recall  f1-score   support

        fair       0.67      0.96      0.79       418
  overpriced       0.09      0.01      0.02       106
 underpriced       0.36      0.05      0.09       100

    accuracy                           0.66       624
   macro avg       0.37      0.34      0.30       624
weighted avg       0.52      0.66      0.55       624


Training: DecisionTree
Accuracy: 0.6041666666666666
              precision    recall  f1-score   support

        fair       0.74      0.76      0.75       418
  overpriced       0.36      0.32      0.34       106
 underpriced       0.26      0.25      0.25       100

    accuracy                           0.60       624
   macro avg       0.45      0.44      0.45       624
weighted avg       0.60      0.60      0.60       624


Training: RandomForest
Accuracy: 0.6426282051282052
              precision    recall  f1-score   support

        fair       

In [19]:
best_model_name = max(results, key=results.get)
best_model_name


'GradientBoosting'

In [20]:
best_model = models[best_model_name]

final_clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", best_model)
])

final_clf.fit(X, y)

joblib.dump(final_clf, "models/overpricing_model.pkl")
print("Saved → overpricing_model.pkl")


Saved → overpricing_model.pkl
